# LNO-type polarisation material analysis
This file will find the sublattices in your image. Check out Atomap.org for lots of information on how to use atomap.

Note: this file is not guaranteed to find what you want. It varies from image
to image. Check out the other "2_Find_Sublattices..." files or the other notebooks for different
types of materials.

In [ ]:
import os
import hyperspy.api as hs
import numpy as np
import atomap.api as am
import temul.api as tml

In [ ]:
# choose your matplotlib backend. qt will plot in a separate window.
%matplotlib nbagg
import matplotlib
matplotlib.pyplot.style.use("grayscale")

## Load the file or some dummy data

Hyperspy can load many different filetypes: dm3, dm4, emd, tif.
Check their website!

In [ ]:
# Example dummy dataset for testing

import temul.dummy_data as dd
sublattice = dd.get_polarised_single_sublattice()
image = sublattice.signal

In [ ]:
''' Set the directory (what folder your data is in!)
    Notice the slashes must be forward slashes. 
    Load your file
'''
# directory = 'C:/Users/Example/notice/the/forward/slashes'
# os.chdir(directory)

# Or load an actual dataset
# image = hs.load('example_awesome_image.tif')

In [ ]:
# plot the image to see how it looks!
image.plot()

In [ ]:
# set the pixel size (called sampling in this toolkit) and the units

sampling = image.axes_manager[-1].scale
units = image.axes_manager[-1].units

print(sampling)  # if sampling is 1, then the image is probably not calibrated.

## Find the atom positions
Get the Pixel separation between the atoms in your first sublattice

Note: If you have used the image filter file, then replace "image" with
"image_ifft" or whatever you called your filtered file.

In [ ]:
# look at the different pixel separations available, this can take some time.

feat = am.get_feature_separation(image, separation_range=(5, 20), pca=True)
feat.plot()

If you're happy with one of the pixel separations, then set it below. If
    you're not, you may need to change your separation_range or filter your
    image with the "1b_Filter_Data.py" file.

Note: If you have used the image filter file, then replace "image" with
"image_ifft" or whatever you called your filtered file.    


In [ ]:
sep = 5  # just an example

atom_positions1 = am.get_atom_positions(image, separation=sep, pca=True)
# save these original sub1 positions!

np.save('atom_positions1.npy', arr=atom_positions1)
# how to reload this file format
# example_positions = np.load('atom_positions1.npy')

### Create the first sublattice, which we call sub1

In [ ]:
sub1 = am.Sublattice(atom_position_list=atom_positions1,
                     image=image, color='red')

sub1.find_nearest_neighbors()
# you can plot the sublattice easily with:
# sub1.plot()

In [ ]:
# You can refine the sublattice easily with a 2D Gaussian or COM algorithm:

sub1.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.2)
# sub1.refine_atom_positions_using_center_of_mass(percent_to_nn=0.2)

np.save('atom_positions1_refined.npy', [sub1.x_position, sub1.y_position])

In [ ]:
# this is needed to create the second sublattice, use atom_plane_tolerance=1 for these very deformed planes

sub1.construct_zone_axes(atom_plane_tolerance=1)
# sub1.plot_planes()  # this can take a long time for large images!

### Create and save the Atom Lattice Object
This contains our two sublattices.

In [ ]:
atom_lattice = am.Atom_Lattice(image=image.data,
                               name='LNO-type structure',
                               sublattice_list=[sub1])

atom_lattice.save(filename="Atom_Lattice.hdf5", overwrite=True)


In [ ]:
atom_lattice.plot()

## Find the polarisation vectors
Now we need to get the (x, y) and (u, v) data for the polarisation vectors.
For the LNO-type material, we need to use just the first sublattice to find the polarisation vectors.

We can use the temul toolkit to do this easily with the save information from above.

In [ ]:
n = 5  # example
zone_axis = 0  # example
x, y, u, v = tml.atom_deviation_from_straight_line_fit(
    sub1, zone_axis, n)


## Plot the polarisation vectors.

Set the parameters below. Don't be overwhelmed, plot_style is the most important input! Many of the rest
you can ignore.

Note: image, sampling, units are loaded from the initial loading step, but you
      can do that here.
Note: u, v, x, y are all loaded from the previous step


In [ ]:
save = None  # save = 'example_name' will save the image
# plot_style options are:
#   'vector', 'colormap', 'contour, 'colorwheel', 'polar_colorwheel'
# for the DPC-style colorwheel just use plot_style = 'polar_colorwheel'

plot_style = 'colormap'
overlay = True  # the vectors will be plotted on the image
unit_vector = False  # formerly called normalise
vector_rep = 'angle'  # 'magnitude' or 'angle'
degrees = False  # Set to True for degrees, False for radians
angle_offset = None
title = ""
color = 'red'  # may be ignored depending on the plot_style
cmap = 'cet_coolwarm'  # may be ignored depending on the plot_style
alpha = 1.0  # transparency of image or vectors, depending on plot_style
image_cmap = 'gray'
ticks = None
scalebar = False
monitor_dpi = None  # set to ~200 to make too-large images a bit smaller
no_axis_info = True  
invert_y_axis = True
antialiased = False  # relevant for the contour mapping
levels = 20  # relevant for the contour mapping
remove_vectors = False
scale = None  # set to 0.001-0.01 to change arrow size
width = None  # set to ~0.005 for chunky (thicker) arrows
minshaft = 1
minlength = 1
headwidth = 3.0
headlength = 5.0
headaxislength = 4.5
quiver_units = 'width'
pivot = 'middle'
angles = 'xy'
scale_units = 'xy'

# plot the vectors!
ax_vectors = tml.plot_polarisation_vectors(
    x, y, u, v, image.data, sampling=sampling, units=units,
    plot_style=plot_style, overlay=overlay, unit_vector=unit_vector,
    vector_rep=vector_rep, degrees=degrees, angle_offset=angle_offset,
    save=save, title=title, color=color, cmap=cmap, alpha=alpha,
    image_cmap=image_cmap, monitor_dpi=monitor_dpi,
    no_axis_info=no_axis_info, invert_y_axis=invert_y_axis, ticks=ticks,
    scalebar=scalebar, antialiased=antialiased, levels=levels,
    remove_vectors=remove_vectors, quiver_units=quiver_units, pivot=pivot,
    angles=angles, scale_units=scale_units, scale=scale, headwidth=headwidth,
    headlength=headlength, headaxislength=headaxislength, width=width,
    minshaft=minshaft, minlength=minlength)

